## US National Census (Income)

*About this Dataset*

**US Adult Census** (1994) relates income to social factors: 

- *age*: continuous.
- *workclass*: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- *fnlwgt*: continuous.
- *education*: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- *education-num*: continuous.
- *marital-status*: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- *occupation*: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- *relationship*: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- *race*: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- *sex*: Female, Male.
- *capital-gain*: continuous.
- *capital-loss*: continuous.
- *hours-per-week*: continuous.
- *native-country*: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

Each row is labelled as either having a salary greater than ">50K" or "<=50K".

Note: This Dataset was obtained from the UCI repository, it can be found on

https://archive.ics.uci.edu/ml/datasets/census+income, http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/

### Preprocessing

In [11]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
np.seterr(divide = 'ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from scipy.stats import kstest

%run utils.py

path = Path(os.getcwd()).parent

columns = ['Age','Workclass','fnlgwt','Education','Education Num','Marital Status',
           'Occupation','Relationship','Race','Sex','Capital Gain','Capital Loss',
           'Hours/Week','Country','Above/Below 50K']

train = pd.read_csv(os.path.join(path, 'data/census_income/adult.data'), names=columns)
test = pd.read_csv(os.path.join(path, 'data/census_income/adult.test'), names=columns)
test = test.iloc[1:] # drop first row from test set

df = pd.concat([train, test])

In [12]:
df.replace(' >50K.', ' >50K', inplace=True)
df.replace(' <=50K.', ' <=50K', inplace=True)

df.dropna()
df.reset_index()

ctg = ['Workclass', 'Sex', 'Education', 'Marital Status', 
       'Occupation', 'Relationship', 'Race', 'Country'] # Categorical to Numerical

for c in ctg:
    df = pd.concat([df, pd.get_dummies(df[c], 
                                       prefix=c,
                                       dummy_na=False)], axis=1).drop([c],axis=1)

df_high = df[df['Above/Below 50K'] == " >50K"].copy(deep=True)
df_low = df[df['Above/Below 50K'] == " <=50K"].copy(deep=True)

### Experiment

- dropping only one instance at a time?
- how to find a good temperature value?
- https://www.sciencedirect.com/science/article/abs/pii/S0167865513000020


In [13]:
# Randomize data
df_low = df_low.reindex(np.random.permutation(df_low.index))
df_high = df_high.reindex(np.random.permutation(df_high.index))

rep = df.copy(deep=True)
non_rep = pd.concat([df_low.head(21000).copy(deep=True),
                     df_high.head(100).copy(deep=True)], sort=True)

print(rep['Above/Below 50K'].value_counts())
print(non_rep['Above/Below 50K'].value_counts())

rep['label'] = 'rep'
non_rep['label'] = 'nonrep'

 <=50K    37155
 >50K     11687
Name: Above/Below 50K, dtype: int64
 <=50K    21000
 >50K       100
Name: Above/Below 50K, dtype: int64


## KS-Test and ROC-curve

The **Kolmogorov-Smirnov test** can be used to test the null hypothesis that the population distribution from which the data sample is drawn is a uniform distribution.

If the P value is small, conclude that the two groups were sampled from populations with different distributions. The populations may differ in median, variability or the shape of the distribution. 

In [14]:
from random import uniform

example_preds = []

for i in range(5000):
    example_preds.append(uniform(0.5, 0.51))

kstest(example_preds, 'uniform')

KstestResult(statistic=0.5000006443892051, pvalue=0.0)

# MRS Algorithm

In [ ]:
data = pd.concat([rep, non_rep], sort=True).copy(deep=True)
data.reset_index(drop=True, inplace=True)
data['probs'] = len(data.label)*[0]

temperature = 1 
max_drop = 50
limit = 20

ks = []
auc = []
ratio = []

while (len(data[data.label == 'nonrep']) > max_drop and
       len(data.label) > limit):

    rf = RandomForestClassifier(n_estimators=100, 
                                bootstrap=True,
                                max_depth=5,
                                oob_score=True)
    
    probs = cross_val_predict(rf,
                              data.drop(['label', 'Above/Below 50K', 'probs'], axis=1),
                              data['label'], 
                              cv=3,
                              method='predict_proba')
    
    preds = cross_val_predict(rf,
                              data.drop(['label', 'Above/Below 50K', 'probs'], axis=1),
                              data['label'], 
                              cv=3)
    
    if preds[0] == 'nonrep' and round(probs[0][0], 0) == 1:
        data['probs'] = [p[1] for p in probs]
    else:
        data['probs'] = [p[0] for p in probs]
    
    drop_id = []
    for _ in range(max_drop):
        softmax = sample(data[data.label == 'nonrep']['probs'], temperature)
        drop = data[data.label == 'nonrep'].iloc[[np.argmax(softmax)]].index[0]
        drop_id.append(drop)
        
    data.drop(data.index[drop_id], inplace=True)
    data.reset_index(drop=True, inplace=True)

    # EVALUATION
    ratio.append(data['Above/Below 50K'].value_counts()[0]/data['Above/Below 50K'].value_counts()[1])
    ks.append(kstest(data.probs, 'uniform'))
    auc.append(metrics.roc_auc_score([1 if k == 'nonrep' else 0 for k in data.label], data.probs))
    print('auc =', metrics.roc_auc_score([1 if k == 'nonrep' else 0 for k in data.label], data.probs))
    print('length of current dataframe:', len(data.label))

auc = 0.5878509637114819
length of current dataframe: 69892
auc = 0.5878944950774216
length of current dataframe: 69842
auc = 0.5873090694567326
length of current dataframe: 69792
auc = 0.5878308240868074
length of current dataframe: 69742
auc = 0.5869877813812993
length of current dataframe: 69692
auc = 0.5871945734361663
length of current dataframe: 69642
auc = 0.5865659996359049
length of current dataframe: 69592
auc = 0.5872994514303936
length of current dataframe: 69542


## Results

**More/Less than 50K**

**Representative Sample:**
- more: 37155
- less: 11687
- 3.18 : 1


**Non-Representative Sample:**
- more: 21000
- less: 11687
- 1.80 : 1


**MRS corrected Sample:**
- more: 6363
- less: 2542
- 2.50 : 1

In [ ]:
plt.plot(auc)

In [ ]:
plt.plot(ratio)

In [ ]:
ks

In [ ]:
plt.plot([a[0] for a in ks])

In [ ]:
data

In [ ]:
 <=50K    37155
 >50K     11687
Name: Above/Below 50K, dtype: int64
 <=50K    21000
 >50K     11687
Name: Above/Below 50K, dtype: int64

In [ ]:
print(data['Above/Below 50K'].value_counts())

In [ ]:
data[data.label == 'nonrep']['Above/Below 50K'].value_counts()